###  1. Importaciones y corpus

Importamos PyTorch y utilidades, y definimos un corpus mínimo para demostrar Word2Vec (Skip‑Gram).


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random   

In [2]:
corpus = [
    "el gato come pescado",
    "el perro come carne",
    "el pájaro vuela alto",
    "el pez nada en el agua"
]

###  2. Preprocesamiento

- Tokenizamos el corpus (separamos palabras)
- Construimos el vocabulario
- Creamos los mapas palabra↔índice (one‑hot implícito)


In [3]:
# Tokenizamos el corpus
tokens = [sentence.split() for sentence in corpus]
vocab = sorted(set(sum(tokens, [])))

# Mapas palabra <-> índice
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(vocab)
print("Vocabulario:", vocab)

Vocabulario: ['agua', 'alto', 'carne', 'come', 'el', 'en', 'gato', 'nada', 'perro', 'pescado', 'pez', 'pájaro', 'vuela']


###  3. Crear dataset con ventanas de contexto (Skip‑Gram)

Ventana de tamaño 2: para cada palabra objetivo, tomamos hasta 2 palabras antes y 2 después como contexto. Generamos pares (target, context).


In [4]:
def generate_skipgram_pairs(tokens, window_size=2):
    pairs = []
    for sentence in tokens:
        for i, target in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    pairs.append((target, sentence[j]))
    return pairs

pairs = generate_skipgram_pairs(tokens, window_size=2)
print("Ejemplo de pares (target, context):", pairs[:10])

Ejemplo de pares (target, context): [('el', 'gato'), ('el', 'come'), ('gato', 'el'), ('gato', 'come'), ('gato', 'pescado'), ('come', 'el'), ('come', 'gato'), ('come', 'pescado'), ('pescado', 'gato'), ('pescado', 'come')]


###  4. Dataset en PyTorch

Empaquetamos los pares (target, context) en un `Dataset` y un `DataLoader` para entrenamiento por lotes.


In [5]:
def generate_skipgram_pairs(tokens, window_size=2):
    pairs = []
    for sentence in tokens:
        for i, target in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    pairs.append((target, sentence[j]))
    return pairs

pairs = generate_skipgram_pairs(tokens, window_size=2)
print("Ejemplo de pares (target, context):", pairs[:10])

Ejemplo de pares (target, context): [('el', 'gato'), ('el', 'come'), ('gato', 'el'), ('gato', 'come'), ('gato', 'pescado'), ('come', 'el'), ('come', 'gato'), ('come', 'pescado'), ('pescado', 'gato'), ('pescado', 'come')]


###  5. Modelo Word2Vec (Skip‑Gram)

Arquitectura mínima:
- Capa de embeddings (convierte índices→vectores)
- Capa lineal de salida (vocabulario) con softmax implícito en la pérdida.


In [6]:
class Word2VecDataset(Dataset):
    def __init__(self, pairs, word_to_idx):
        self.pairs = pairs
        self.word_to_idx = word_to_idx

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        target, context = self.pairs[idx]
        return torch.tensor(self.word_to_idx[target]), torch.tensor(self.word_to_idx[context])

dataset = Word2VecDataset(pairs, word_to_idx)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

### ⚙️ 6. Entrenamiento

Optimizamos con `CrossEntropyLoss` y `Adam`. El objetivo es predecir la palabra de contexto dada la palabra objetivo.


In [7]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, target_word):
        emb = self.embeddings(target_word)
        out = self.output(emb)
        return out

###  7. Ver los embeddings finales

Inspeccionamos la matriz de embeddings (una fila por palabra). Palabras con contextos similares tendrán vectores cercanos.


In [8]:
embedding_dim = 8
model = Word2VecModel(vocab_size, embedding_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 200

for epoch in range(epochs):
    total_loss = 0
    for target, context in dataloader:
        optimizer.zero_grad()
        output = model(target)
        loss = criterion(output, context)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 40 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 40/200, Loss: 9.1965
Epoch 80/200, Loss: 8.4766
Epoch 120/200, Loss: 8.3430
Epoch 160/200, Loss: 8.2753
Epoch 200/200, Loss: 8.2674


###  En General

- La ventana de contexto se desplaza palabra por palabra.
- Generamos pares (target, contexto) para aprendizaje supervisado.
- El modelo aprende a predecir el contexto a partir de la palabra central.
- Los pesos de la capa de embeddings son las representaciones vectoriales (significado) de las palabras.


In [9]:
embeddings = model.embeddings.weight.data
for word, idx in word_to_idx.items():
    print(f"{word:10s} -> {embeddings[idx].tolist()}")

agua       -> [0.35946959257125854, 0.23149815201759338, 1.0404404401779175, -0.2307528406381607, -1.9380111694335938, -1.3543272018432617, 2.47517466545105, 2.420862913131714]
alto       -> [1.2128000259399414, -2.3942513465881348, -0.301423043012619, -1.10421884059906, 2.416069984436035, 1.3163093328475952, -0.593668520450592, 2.4861366748809814]
carne      -> [-3.0026299953460693, 2.461963415145874, 0.3777582347393036, -2.414140462875366, 1.2446975708007812, -0.6849075555801392, -0.4919763505458832, -0.2393103688955307]
come       -> [-2.616908550262451, 0.35495927929878235, -0.15568451583385468, -0.6339329481124878, -1.7604267597198486, 1.0748422145843506, 2.94465970993042, -1.3403608798980713]
el         -> [-0.2979934513568878, -2.603065013885498, 1.8233269453048706, 0.16880477964878082, 2.2370550632476807, -1.4897934198379517, 0.5872586965560913, -0.6455591320991516]
en         -> [-0.4081358015537262, -0.3527601957321167, -0.39999285340309143, 2.420285940170288, -1.480732083320